# Eigenmode Analysis of an Optical Waveguide Using a Yee-Grid Discretization

This notebook demonstrates how to compute the eigenmodes of an optical waveguide using a Yee-mesh discretization. In this formulation (``wgmodes_yee``), the two transverse magnetic field components are calculated explicitly, and the remaining four electromagnetic fields are represented on a staggered grid. This approach mirrors the spatial staggering used in finite-difference time-domain (FDTD) methods and provides an alternative discretization to node-centered formulations. The discretization follows early finite-difference eigenmode approaches based on staggered grids, as described originally in the work of Zhu and Brown (2002).

**Full-vectorial finite-difference analysis of microstructured optical fibers**<br>
Z. Zhu and T. G. Brown<br>
*Optics Express*, **10**(17) 853 (2002).  
<https://doi.org/10.1364/OE.10.000853>

The example calulates the fundamental TE mode of a typical x-cut lithium-niobate on insulator ridge waveguide at 1550 nm.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%load_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
n1 = 1.46           # SiO2 lower cladding
n2o = 2.21          # Lithium niobate (ordinary)
n2e = 2.14          # Lithium niobate (extraordinary)
n3 = 1.46           # SiO2 upper cladding

# Vertical dimensions:
h1 = 500            # lower cladding (nm), will be stretched
h2 = 700            # silicon core (nm)
h3 = 500            # upper cladding (nm), will be stretched
rh = 400            # ridge height

# Horizontal dimensions:
w = 900             # full rib width (nm)
side = 800          # Space on side (nm), will be stretched

# Grid size:
dx = 5              # grid size (x)
dy = dx             # grid size (y)

# Build waveguide mesh (epsxx, epsyy, epszz)
x,y,xc,yc,nx,ny,epsxx,edges = mode.waveguidemesh([n1,n2e,n3],[h1,h2,h3],rh,w/2,side,dx,dy,return_edges=True)
epsyy = epsxx.copy()
epsyy[np.isclose(epsxx, n2e**2)] = n2o**2
epszz = epsyy

In [ ]:
# Stretch cladding layers (NORTH, SOUTH, EAST) by 1.5X
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[100,100,160,0],[2,2,2,1])

In [ ]:
wavelength = 1550   # vacuum wavelength (nm)
nmodes = 1          # number of modes to compute

# First, consider the fundamental TE mode:
neff, Ex, Ey, Ezj, Hx, Hy, Hzj = mode.wgmodes_yee(wavelength,n2e,nmodes,dx,dy,
                                                  epsxx=epsxx,epsyy=epsyy,epszz=epszz,boundary='000E')
print(f"neff = {neff:.6f}")

# Rescale electric fields for plotting:
# By default, E-fields are normalized by the vacuum impedance η₀.
# Here we multiply by n2e to normalize instead by the core impedance η₂ = η₀/n₂.
Ex *= n2e
Ey *= n2e
Ezj *= n2e


In [ ]:
# Make a table showing the dimensions of each calculated field component on the Yee grid
(nx, ny) = epsxx.shape
fields = {
    "eps" : epsxx,
    "Ex"  : Ex,
    "Ey"  : Ey,
    "Ezj" : Ezj,
    "Hx"  : Hx,
    "Hy"  : Hy,
    "Hzj" : Hzj,
}

print(f"Grid parameters: nx = {nx}, ny = {ny}\n")

for name, F in fields.items():
    print(f"{name:<5} : {str(F.shape):>12}")

In [ ]:
# Plot all 6 field components, with common axes

plots = [
    dict(field=Ex,  title="Ex",   x=xc,  y=y, ),
    dict(field=Ey,  title="Ey",   x=x,  y=yc,),
    dict(field=Ezj, title="Ez*j", x=x,  y=y, ),
    dict(field=Hx,  title="Hx",   x=x,  y=yc,),
    dict(field=Hy,  title="Hy",   x=xc, y=y, ),
    dict(field=Hzj, title="Hz*j", x=xc, y=yc,),
]

fig, axes = plt.subplots(2,3,sharex=True,sharey=True,figsize=(10,10))
fig.subplots_adjust(wspace=0, hspace=0)

for ax, p in zip(axes.flat, plots):
    ax.pcolormesh(p["x"], p["y"], np.abs(p["field"]), cmap="jet", vmax=1, shading="auto")
    ax.set_aspect("equal")
    ax.text(
        0.95, 0.95, p["title"],
        transform=ax.transAxes,
        color="white", weight="bold", fontsize=14,
        ha="right", va="top"
    )
    ax.add_collection(LineCollection(edges, colors="black", linewidths=1))

plt.show()

## Collocated Fields

By default, `wgmodes_yee` returns fields on a staggered Yee grid, where each field component is sampled at a different spatial location. For applications requiring all fields at the same grid points, use the `collocate=True` option to linearly interpolate all field components to cell centers.

In [ ]:
# Calculate fields with collocate=True
neff, Ex, Ey, Ezj, Hx, Hy, Hzj = mode.wgmodes_yee(wavelength, n2e, nmodes, dx, dy,
                                                  epsxx=epsxx, epsyy=epsyy, epszz=epszz,
                                                  boundary='000E', collocate=True)

# Show dimensions of each field component (now all identical)
(ny, nx) = epsxx.shape
fields = {
    "eps" : epsxx,
    "Ex"  : Ex,
    "Ey"  : Ey,
    "Ezj" : Ezj,
    "Hx"  : Hx,
    "Hy"  : Hy,
    "Hzj" : Hzj,
}

print(f"Grid parameters: nx = {nx}, ny = {ny}\n")

for name, F in fields.items():
    print(f"{name:<5} : {str(F.shape):>12}")